# Human metabolic model
Recon3D_3.01 downloaded from https://vmh.uni.lu/#downloadview 


In [2]:
# -*- coding:utf-8 -*-
import cobra
import glob
from tqdm import tqdm
from collections import defaultdict

In [5]:
# dictionary of the in and out metabolites and their fluxes for each species in the model
# species:
#   metabolite:
#     flux
human_met = {}
human_model_file = "agora-1_02/human-model/Recon3D_301_SBML/Recon3D_301.xml"
model = cobra.io.read_sbml_model(human_model_file)
# first get the solvers output of the model
solution = model.optimize()

# for each of the "exchange" reactions, find the incoming and outgoing fluxes 
# using the initial stoichiometry matrix value multiplied by the solutions flux for that reaction
# For example, a metabolite whose initial stoich value was -1 but has a flux value of -5 would be incoming.
fluxes = {}
for rxn in model.exchanges:
    for met in rxn.metabolites:
        stoich = rxn.metabolites[met]
        flux = stoich * solution.fluxes[rxn.id]
        fluxes[met.name] = flux
#in_fluxes = [m for m in fluxes if fluxes[m] > 0]
#out_fluxes = [m for m in fluxes if fluxes[m] < 0]
#print("%d in fluxes, %d out fluxes" % (len(in_fluxes), len(out_fluxes)))
in_out_fluxes = {m: fluxes[m] for m in fluxes if fluxes[m] != 0}
## the model id is the ID of the strain
#species_io_met[model.id] = fluxes

human_met = fluxes

In [6]:
# write them to a file
human_fluxes_file = "agora-1_02/human-model/fluxes.tsv"
with open(human_fluxes_file, 'w') as out:
    out.write(''.join("%s\t%s\n" % (m, str(f)) for m, f in human_met.items()))